
# ChatGPT Prompt Engineering for Developers.  
  
...
  
**Content**
1. [Introduction(https://learn.deeplearning.ai/courses/chatgpt-prompt-eng/lesson/1/introduction)
2. Guidelines for Prompting

##1. Guidelines for Prompting

In [27]:
!pip install openai

In [5]:
# import libraries and set api key

import openai
import os

# os.environ['OPENAI_API_KEY'] = "sk-" # Replace ... with your key and set as os variable
openai.api_key = os.getenv('OPENAI_API_KEY')

In [28]:
print(openai.__version__)

1.35.13


In [29]:
# create helper function to use propmpts and observer the generated output.
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

###**Prompting Principles**  
Principle 1: Write clear and specific instructions    
Principle 2: Give the model time to "think"  
  
####**Princple 1: Write clear and specific instructions**  

####**Tactics**  
**Tactic 1:** Use delimiters to indicate clearly distinct parts of input. Delimiters can be e.g ''', <<<>>>, `<tag> </tag>`, :.

In [8]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}
"""
response = get_completion(prompt)
print(response)

Providing clear and specific instructions to a model will guide it towards the desired output and reduce the chances of receiving irrelevant or incorrect responses, with longer prompts often providing more clarity and context for more detailed and relevant outputs.


**Tactic 2:** Ask for structured output.  
E.g JSON, HTML.

In [9]:
prompt = f"""
Generate a list of three made-up movie titles along \
with their directors and genres.
Provide them in JSON format with the following keys:
movie_id, title, director, genre.
"""
response = get_completion(prompt)
print(response)

[
    {
        "movie_id": 1,
        "title": "Galactic Odyssey",
        "director": "Aurora Steele",
        "genre": "Sci-Fi"
    },
    {
        "movie_id": 2,
        "title": "Midnight Masquerade",
        "director": "Julian Black",
        "genre": "Romance"
    },
    {
        "movie_id": 3,
        "title": "Shadow Realm",
        "director": "Elena Nightshade",
        "genre": "Horror"
    }
]


In [15]:
type(response)

str

In [21]:
# Store the response string as a list of dicts
import json
movies: list = json.loads(response)
print(type(movies))
print(json.dumps(movies, indent=2))

<class 'list'>
[
  {
    "movie_id": 1,
    "title": "Galactic Odyssey",
    "director": "Aurora Steele",
    "genre": "Sci-Fi"
  },
  {
    "movie_id": 2,
    "title": "Midnight Masquerade",
    "director": "Julian Black",
    "genre": "Romance"
  },
  {
    "movie_id": 3,
    "title": "Shadow Realm",
    "director": "Elena Nightshade",
    "genre": "Horror"
  }
]


**Tactis 3:** Ask model to check if conditions are satisfied

In [22]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
…
Step N - ...

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print(response)

Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [23]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


**Tactic 4:** Few-shot prompting

In [24]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The tallest trees weather the strongest storms; the brightest stars shine in the darkest nights; the strongest hearts are forged in the hottest fires.


####**Princple 2: Give the model time to think**  
####**Tactics**  

**Tactic 1:** Specify the steps required to complete the task  

In [25]:
text = f"""
In a charming village, siblings Jack and Jill set out on\
 a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""

prompt = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt)
print(response)

1 - Jack and Jill go on a quest to fetch water from a well, but encounter misfortune on the way back home.

2 - Jack et Jill partent en quête d'eau d'un puits, mais rencontrent un malheur sur le chemin du retour.

3 - Jack, Jill

4 - {
    "summary": "Jack and Jill go on a quest to fetch water from a well, but encounter misfortune on the way back home.",
    "names": ["Jack", "Jill"]
}


**Ask for output in a specified format.**

In [26]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, two siblings, go on a quest to fetch water from a well on a hill, but encounter misfortune along the way.

Translation: Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits sur une colline, mais rencontrent des malheurs en chemin.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Jack et Jill, deux frères et sœurs, partent en quête d'eau d'un puits sur une colline, mais rencontrent des malheurs en chemin.",
  "num_names": 2
}


**Topic 2:** Instruct the model to work out its own solution before rushing into a conclusion.

In [30]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.


Note that the student´s solution is acutally incorrect.  
  
Let´s fix this by instructing the model to work out its own solution first.

In [32]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost: $100 * x + $250 * x + $100,000 + $10 * x = $360 * x + $100,000

The total cost for the first year of operations as a function of the number of square feet is $360x + $100,000.
```
Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
Incorrect


###**Model Limitations: Hallucinations.**  
Boie is a real company, the product name is not real.

In [41]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush designed to provide a superior cleaning experience. It features ultra-soft bristles that are gentle on the gums and teeth, while still effectively removing plaque and debris. The toothbrush also has a slim design that makes it easy to maneuver and reach all areas of the mouth.

One of the standout features of the AeroGlide UltraSlim Smart Toothbrush is its smart technology. It connects to a mobile app that tracks your brushing habits and provides personalized recommendations for improving your oral hygiene routine. The app also includes a timer and pressure sensor to ensure you are brushing for the recommended two minutes and not applying too much pressure.

Overall, the AeroGlide UltraSlim Smart Toothbrush by Boie is a sleek and innovative toothbrush that can help you achieve a cleaner and healthier smile.


In [42]:
len(response)

In [43]:
# modify response of +1k chars split into a more read friendly print out with new row after every n char.

index = 100
while index < len(response):
  response = response[:index] + "\n" + response[index:]
  index += 101 # Add 101 to account for the newline character
print(response)


The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush designed to provide a sup
erior cleaning experience. It features ultra-soft bristles that are gentle on the gums and teeth, wh
ile still effectively removing plaque and debris. The toothbrush also has a slim design that makes i
t easy to maneuver and reach all areas of the mouth.

One of the standout features of the AeroGlide 
UltraSlim Smart Toothbrush is its smart technology. It connects to a mobile app that tracks your bru
shing habits and provides personalized recommendations for improving your oral hygiene routine. The 
app also includes a timer and pressure sensor to ensure you are brushing for the recommended two min
utes and not applying too much pressure.

Overall, the AeroGlide UltraSlim Smart Toothbrush by Boie 
is a sleek and innovative toothbrush that can help you achieve a cleaner and healthier smile.


#### A note about the backslash
- In the notebook, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.